In [1]:
# Importing the dependencies.
import numpy as np
from keras.layers import Dense, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd

Using TensorFlow backend.


In [2]:
df= pd.read_csv('training_data.csv')
print(df.shape)
print(df.tail())

dataset = np.array(df)
np.random.shuffle(dataset)
x = dataset
y = dataset
x = x[:, 1:2501]
y = y[:,0]

(13199, 2501)
       1  0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  ...  0.1509  0.1510  \
13194  9  0    0    0    0    0    0    0    0    0  ...       0       0   
13195  9  0    0    0    0    0    0    0    0    1  ...       0       0   
13196  9  0    0    0    0    0    0    0    0    0  ...       0       0   
13197  9  0    0    0    0    0    0    0    0    0  ...       0       0   
13198  9  0    0    0    0    0    0    0    0    0  ...       0       0   

       0.1511  0.1512  0.1513  0.1514  0.1515  0.1516  0.1517  0.1518  
13194       0       0       0       0       0       0       0       0  
13195       0       0       0       0       0       0       0       0  
13196       0       0       0       0       0       0       0       0  
13197       0       0       0       0       0       0       0       0  
13198       0       0       0       0       0       0       0       0  

[5 rows x 2501 columns]


In [3]:
x_train = x[0:12000,:]
y_train = x_train/255. #Normalization.
x_test = x[12000:13201 , :]
y_test = x_test/255. #Normalization.

In [4]:
y = y.reshape(y.shape[0],1)
y_train = y[0:12000, :]
print(y_train)
y_train = y_train.T
y_test = y[12000:13201, :]
y_test = y_test.T

[[2]
 [2]
 [8]
 ...
 [2]
 [5]
 [2]]


In [5]:
print('Number of training examples =',x_train.shape[0])
print('Number of test examples =',y_train.shape[0])
print('x_train shape =',x_train.shape)
print('y_train shape =',y_train.shape)
print('x_test shape =',x_test.shape)
print('y_test shape =',y_test.shape)


Number of training examples = 12000
Number of test examples = 1
x_train shape = (12000, 2500)
y_train shape = (1, 12000)
x_test shape = (1199, 2500)
y_test shape = (1, 1199)


In [6]:
image_x =50
image_y =50

train_y = np_utils.to_categorical(y_train)
test_y =np_utils.to_categorical(y_test)
train_y = train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1],test_y.shape[2])
x_train = x_train.reshape(x_train.shape[0], 50, 50, 1)
x_test = x_test.reshape(x_test.shape[0], 50, 50, 1)

print('x_train shape =',x_train.shape)
print('x_test shape =',x_test.shape)
print('train_y shape =',train_y.shape)

x_train shape = (12000, 50, 50, 1)
x_test shape = (1199, 50, 50, 1)
train_y shape = (12000, 12)


In [8]:
def keras_model(image_x,image_y):
    num_of_classes = 12
    model = Sequential()
    model.add(Conv2D(32, (5,5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2) ,strides=(2,2), padding='same'))
    model.add(Conv2D(64, (5,5), activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(2,2) ,strides=(2,2), padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = 'face_rec_256.h5'
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]


    return model, callbacks_list

In [9]:
model, callbacks_list = keras_model(image_x,image_y)
model.fit(x_train, train_y, validation_data =(x_test, test_y),
          epochs=1, batch_size=64, callbacks=callbacks_list)
scores = model.evaluate(x_test, test_y, verbose=0)
print('CNN Error :%.2f%%'%(100-scores[1]*100))
print_summary(model)

model.save('HandEmo.h5')

Train on 12000 samples, validate on 1199 samples
Epoch 1/1
12000/12000 [==============================] - 123s 10ms/step - loss: 0.4079 - accuracy: 0.9448 - val_loss: 5.0856e-04 - val_accuracy: 1.0000


C:\Users\jigyasa\Documents\satyam anaconda\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


CNN Error :0.00%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              6554624   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1